In [136]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
必要なライブラリをインポートする

In [137]:
# データベース設定
DB_FILE = "kyujin.db"

# SQLite 初期化
def initialize_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS kyujin (
            JobCategory TEXT,
            Industry TEXT,
            KeyRequirements TEXT,
            OpenPosition TEXT,
            Features TEXT,
            EmploymentType TEXT,
            SalaryRange TEXT,
            Location TEXT,
            PRIMARY KEY (JobCategory, Industry, Location)
        )
    """)
    conn.commit()
    conn.close()

# 求人情報を保存する
def save_to_db(job_category, industry, key_requirements, open_position, features, employment_type, salary_range, location):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT OR REPLACE INTO kyujin (JobCategory, Industry, KeyRequirements, OpenPosition, Features, EmploymentType, SalaryRange, Location)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (job_category, industry, key_requirements, open_position, features, employment_type, salary_range, location))
    conn.commit()
    conn.close()

# 年収データを抽出する
def extract_salary(soup):
    salary_options = soup.select("select[name='annualsalary'] option")
    salary_values = []
    for option in salary_options:
        value = option.get("value")
        if value and value.isdigit():
            salary_values.append(int(value))
    return salary_values

# 求人情報をスクレイピングする
def scrape_jobs(html_data):
    soup = BeautifulSoup(html_data, "html.parser")

    # 職種
    job_category = soup.select_one("input[name='occupation[]']:checked + label").text.strip()

    # 業種
    industry = soup.select_one("input[name='industry[]']:checked + label").text.strip()

    # 特長＆TOPICS
    features = ", ".join([label.text.strip() for label in soup.select("input[name='topicsid[]']:checked + label")])

    # 雇用形態
    employment_type = ", ".join([label.text.strip() for label in soup.select("input[name='employmenttypeid[]']:checked + label")])

    # 年収範囲（最小と最大を取得して範囲として保存）
    salary_values = extract_salary(soup)
    salary_range = f"{min(salary_values)}万円～{max(salary_values)}万円" if salary_values else "N/A"

    # 勤務地
    location = soup.select_one("input[name='areaid[]']:checked + label").text.strip()

    # データ保存
    save_to_db(job_category, industry, None, None, features, employment_type, salary_range, location)

# 条件でフィルタリングする
def fetch_filtered_kyujin(keywords):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # SQLクエリで条件を適用
    query = "SELECT * FROM kyujin WHERE " + " AND ".join(f"Features LIKE '%{kw}%'" for kw in keywords)
    cursor.execute(query)

    results = cursor.fetchall()
    conn.close()
    return results

kyujin.dbという名前のデータベースを作成する。そこに今回スクレイピングした職種、業種、特長＆TOPICS、雇用形態、年収範囲、勤務地の情報を保存させる。分析に使用するデータを構築したDBに対してクエリを発行させる。

In [140]:
# 年収の正規化関数
def normalize_salary(salary_text):
    try:
        # 「年収400万円～500万円」形式の処理
        salary_text = salary_text.replace("年収", "").replace("万円", "").strip()
        if "～" in salary_text:
            min_salary, max_salary = map(int, salary_text.split("～"))
            return (min_salary + max_salary) // 2  # 範囲の平均を返す
        elif salary_text.isdigit():
            return int(salary_text)
        else:
            return None
    except (ValueError, IndexError):
        return None

# 求人情報の年収を取得する関数
def scrape_job_salaries(base_url, total_pages):
    all_salaries = []

    for page in range(1, total_pages + 1):
        url = f"{base_url}&pagenum={page}"
        print(f"Scraping page {page}: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 年収情報を取得（適切なセレクタを確認）
        salary_elements = soup.select("p.salary")  # 年収が表示されているHTML要素のセレクタ
        for element in salary_elements:
            salary_text = element.text.strip()
            normalized_salary = normalize_salary(salary_text)
            if normalized_salary:
                all_salaries.append(normalized_salary)

        # サーバー負荷を避けるために少し待機
        time.sleep(1)

    return all_salaries

# 平均年収を計算する関数
def calculate_average_salary(salaries):
    if len(salaries) == 0:
        return 0
    return sum(salaries) / len(salaries)

# メイン処理
if __name__ == "__main__":
    # ベースURL
    BASE_URL = "https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000"

    # ページ数を指定（323件の求人情報、1ページ50件表示として計算）
    TOTAL_PAGES = 7

    # 年収データを取得
    all_salaries = scrape_job_salaries(BASE_URL, TOTAL_PAGES)

    # 年収データの確認
    print(f"取得した年収データ: {all_salaries}")

    # 平均年収を計算
    average_salary = calculate_average_salary(all_salaries)
    print(f"\n平均年収: {average_salary:.2f}万円")

Scraping page 1: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=1
Scraping page 2: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=2
Scraping page 3: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=3
Scraping page 4: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=4
Scraping page 5: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=5
Scraping page 6: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=6
Scraping page 7: https://employment.en-japan.com/search/search_list/?aroute=2&topicsid=201&industry=1100&areaid=23&occupation=400000&pagenum=7

勤務地を東京にし、希望する職種はIT・通信関連で週休２日を条件として指定されたURLから求人方法を取得し、正規化されたデータがall_salariesに格納される。それに該当する年収を抽出し、返金年収を抽出しようとした。